<a href="https://colab.research.google.com/github/remre/StriveSchool-ai/blob/main/week5featureeng/day6mnist/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report  
from numpy.core.records import array
import sys
import os
import csv
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier

In [ ]:
arr = np.array(img , dtype=float)

In [ ]:
img.show()

In [3]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = ("/content")

In [5]:
!kaggle datasets download -d scolianni/mnistasjpg

 98% 67.0M/68.4M [00:00<00:00, 61.3MB/s]
100% 68.4M/68.4M [00:00<00:00, 78.2MB/s]


In [ ]:
!unzip "/content/mnistasjpg.zip" -d "/content"

In [ ]:
def createFileList(myDir, format='.jpg'):
    fileList = []
    print(myDir)
    for root, dirs, files in os.walk(myDir, topdown=False):
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
    return fileList

# load the original image
myFileList = createFileList('/content/trainingSet/trainingSet')

for file in myFileList:
    print(file)
    img_file = Image.open(file)
    # img_file.show()

    # get original image parameters...
    width, height = img_file.size
    format = img_file.format
    mode = img_file.mode

    # Make image Greyscale
    #img_grey = img_file.convert('L')
    #img_grey.save('result.png')
    #img_grey.show()

    # Save Greyscale values
    value = np.asarray(img_file.getdata(), dtype=np.int).reshape((img_file.size[1], img_file.size[0]))
    value = value.flatten()
    print(value)
    with open("mnisttraining.csv", 'a') as f:
        writer = csv.writer(f)
        writer.writerow(value)

In [10]:
df = pd.read_csv("mnisttraining.csv")

In [14]:
dff = pd.read_csv("fashion-mnist_test.csv")

In [12]:
dfk = pd.read_csv('mnisttest.csv')

In [13]:
df.head()

,0,10,0.1,0.2,6,0.3,0.4,0.5,0.6,16,12,0.7,5,9,0.8,6.1,1,0.9,3,0.10,0.11,2,0.12,0.13,0.14,0.15,0.16,0.17,0.18,2.1,0.19,0.20,4,0.21,0.22,0.23,12.1,4.1,0.24,16.1,...,0.444,0.445,0.446,0.447,0.448,0.449,0.450,0.451,0.452,0.453,0.454,0.455,0.456,0.457,0.458,0.459,0.460,0.461,0.462,0.463,0.464,0.465,0.466,0.467,0.468,0.469,0.470,0.471,0.472,0.473,0.474,0.475,0.476,0.477,0.478,0.479,0.480,0.481,0.482,0.483
0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,4,0,0,13,0,3,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,3,8,0,1,0,0,9,0,0,0,1,0,0,3,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,5,2,0,0,0,0,0,0,0,0,1,0,0,0,2,4,0,1,10,5,0,0,1,0,0,0,0,0,0,0,2,3,3,2,2,2,4,9,12,10,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,3,6,0,0,2,0,0,3,0,0,6,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,3,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,7,2,2,1,0,2,0,0,6,0,6,8,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [57]:
dff.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,0,0,34,29,7,0,11,24,0,0,3,3,1,0,1,1,0,0,0,0,0,4,0,0,1,0,0,0,0,0,...,242,245,224,245,234,0,3,0,0,0,0,0,0,0,0,0,0,0,0,37,69,94,123,127,138,138,142,145,135,125,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,209,190,181,150,170,193,180,219,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,...,248,238,80,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,174,233,155,0,65,235,216,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,17,0,0,0,0,0,0,0,0,12,94,68,14,0,0,0,0,0,0,0,0,0,0,0,38,106,94,89,94,68,...,0,0,0,0,0,0,87,12,26,0,0,0,0,0,0,0,46,50,29,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,161,212,138,150,169,164,176,202,255,183,26,0,0,0,0,0,0,0,0,0,0,0,1,0,0,129,221,255,255,215,...,255,255,254,252,255,101,196,254,252,100,0,0,0,0,0,168,232,225,0,41,172,147,148,153,155,146,137,141,143,137,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,0,37,0,0,0,0,0,0,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21,204,235,...,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,69,0,2,0,0,0,0,0,0,0,0,0,0


In [15]:
dff["label"].value_counts()

7    1000
6    1000
5    1000
4    1000
3    1000
2    1000
9    1000
1    1000
8    1000
0    1000
Name: label, dtype: int64

In [17]:
X = dff.iloc[:,1:] # iloc ensures X_tr will be a dataframe
y = dff.iloc[:, 0]
X_test = dfk.iloc[0][0]

In [18]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2, random_state=30, stratify=y)

In [72]:
steps = [('scaler', StandardScaler()), ('SVM', SVC(kernel='poly'))]
pipeline = Pipeline(steps) # define Pipeline object

In [22]:
pipe_LogReg = Pipeline([('scaler',StandardScaler()),
                        ('LogisticRegression',LogisticRegression())])

pipe_knn = Pipeline([('scaler',StandardScaler()),
                        ('KNeighborsClassifier',KNeighborsClassifier())])

pipe_GBM = Pipeline([('scaler',StandardScaler()),
                        ("GBM", GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0),)])

pipe_RForest = Pipeline([('scaler',StandardScaler()),
                  ('RandomForestClassifier',RandomForestClassifier())])


pipe_Ada = Pipeline([('scaler',StandardScaler()),
                  ('AdaBoostClassifier',AdaBoostClassifier())])

In [25]:
pipelines = [pipe_LogReg ,pipe_knn,pipe_GBM,pipe_RForest,pipe_Ada]

In [26]:
for pipe in pipelines:
    pipe.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [82]:
model_name =[]
scores=[]

for i,model in enumerate(pipelines):
    pred = model.predict(X_val)
    score = accuracy_score(y_val,pred)
    model_name.append(model.steps[1][0])
    scores.append(score)
    print(i,model.steps[1][0].upper(),':',score)
    print('\n')

0 LOGISTICREGRESSION : 0.806


1 KNEIGHBORSCLASSIFIER : 0.8275


2 GBM : 0.665


3 RANDOMFORESTCLASSIFIER : 0.859


4 ADABOOSTCLASSIFIER : 0.3045


